In [ ]:
# pip install hdbcli

In [ ]:
import platform
from hdbcli import dbapi
import pandas as pd

In [ ]:
conn = dbapi.connect(
    # key='USER1UserKey', 
    address='10.130.222.132',
    port='34216',
    user='PROCESSAND',
    password='Delhi-123-cocoon-jape',
    encrypt=True, 
    sslValidateCertificate=False 
)
cursor = conn.cursor()

In [ ]:
def show_duplicates(schema, table, code_only = False, use_csv_path = None ,sep=","):
    print(f"-- Schema name: {schema}, Table name: {table}")
    
    # code_only: To only print out the SQL code
    # use_csv_path: Enter the file path of DD03M.csv
    # sep: Delimiter of the csv file 
    ## DD03M  csv file must include columns: TABNAME, FIELDNAME, KEYFLAG

    keys = []
    sql1 = f"SELECT DISTINCT FIELDNAME FROM {schema}.DD03M WHERE KEYFLAG='X' AND TABNAME = '{table}';"

    if code_only:
        # Import DD03M from csv file
        if use_csv_path is not None:
            df1 =pd.read_csv(use_csv_path, sep=sep)
            csv_keys = df1.loc[(df1['TABNAME']==table) & (df1['KEYFLAG']=="X")]['FIELDNAME'].unique()
            for key in csv_keys:
                keys.append(str(key))
            if len(keys)==0:
                raise Exception("Error: Table or column missing in the csv file.")
        else:
            raise Exception("Error: Please enter csv file path for offline use.")
    # Import DD03M from shema
    else:        
        cursor.execute(sql1)
        for key in cursor:
            keys.append(str(key[0]))
  
    print("-- Primary keys: " + ",".join(keys))
    _case_key = '||'.join(keys)

    # Count duplicated cases in the target table
    sql2 = f"SELECT COUNT(1) - COUNT(DISTINCT {_case_key}) FROM {schema}.{table};"
    if code_only:
        print(f"--Calculate duplicates for {schema}.{table}")
        print(sql2)
    else:
        cursor.execute(sql2)
        if cursor is None :
            print("No Duplicates")
        else: 
            print("Duplicated rows count: " + str(cursor.fetchall()[0][0]))         

In [ ]:
def show_duplicates_multi_tables(schemas:list, tables:list, code_only = None, use_csv_path = None, sep = ","):
    for schema in schemas:
        for table in tables:
            try:
                show_duplicates(schema,table,code_only,use_csv_path,sep)
                print("")
            except Exception as e:
                print(e)

In [ ]:
def remove_dulplicates(target_schema, source_schema, table, keys:list, sorting_col:list, code_only=False):
    
    key_str = ','.join(keys)
    sorting_str = ",".join(sorting_col)
    print(f"--Target schema: {target_schema}")
    print(f"--Source schema: {source_schema}")
    print(f"--Table: {table}")
    print("--Primary keys: " + key_str)
    print("--Sorting columns: " + sorting_str)

    # # Drop the BACKUP table alreaydy exists
    # drop_if_exist = f"CALL {target_schema}.DROP_IF_EXISTS('{target_schema}', '{table}_BACKUP');"
    
    # Copy and create the backup table and add ROW_NUMBER()
    
    create_table = f'CREATE TABLE "{target_schema}"."{table}_BACKUP" AS (SELECT ROW_NUMBER() OVER(PARTITION BY {key_str} ORDER BY {sorting_str}) AS _ROW_NUMBER, *  FROM {source_schema}.{table})'

    count_rows = f'SELECT _ROW_NUMBER, COUNT(1) FROM "{target_schema}"."{table}_BACKUP" GROUP BY _ROW_NUMBER'

    remove_rows = f'DELETE FROM "{target_schema}"."{table}_BACKUP" WHERE _ROW_NUMBER >= 2'
    
    if code_only:
        # print("-- Drop BACKUP table if exists")
        # print(drop_if_exist)
        print("-- Create BACKUP table in the target schema")
        print(create_table)
        print("-- Duplicated rows will have _ROW_NUMBER >= 2")
        print(count_rows)
        print("-- Remove duplicated rows from the BACKUP table")
        print(remove_rows)
    else:
        # cursor.execute(drop_if_exist)    
        cursor.execute(create_table)
        cursor.execute(count_rows)

        duplicated_rows = 0
        for row in cursor.fetchall()[1:]:
            duplicated_rows += row[1]
        print(f"Schema: {source_schema}, Table: {table}, Duplicated rows: " + str(row[1]))

        # Remove duplicate records from the backup table
        cursor.execute(remove_rows)  
        print(f"Duplicates removed in {target_schema}.{table}_BACKUP")

In [ ]:
#Example 1
show_duplicates_multi_tables(['ABAP_P60', 'P1A_CONSOLIDATED'], tables=['EKKO','EKPO'])

In [ ]:
#Example 2
show_duplicates_multi_tables(['ABAP_P60', 'P1A_CONSOLIDATED'], tables=['EKKO','EKPO'], code_only=True)

In [ ]:
show_duplicates_multi_tables(['ABAP_P60', 'P1A_CONSOLIDATED'], tables=['EKKO'], code_only=True, use_csv_path = "C:/Users/KencoWu/Downloads/results_1.csv", sep = "\t")

In [ ]:
#Example
show_duplicates_multi_tables(['ABAP_P60', 'P1A_CONSOLIDATED'], tables=['EKKO','EKPO'], code_only=False)

In [ ]:
#Example 4
remove_dulplicates("PROCESSAND", "ABAP_P60", "EKKO", ["MANDT", "EBELN"], ["AEDAT", "BUKRS"], code_only=False)

In [ ]:
#Example 5
remove_dulplicates("PROCESSAND", "ABAP_P60", "EKKO", ["MANDT", "EBELN"], ["AEDAT", "BUKRS"], code_only=True)